# LangChain in Practice 🦜

A framework for developing applications powered by large language models (LLMs).

## Multiple Packages in LangChain

- langchain-core: Package contains base abstractions that the rest of the LangChain ecosystem uses and is installed automatically when installing LangChain

- langchain-community: Package contains third-party integrations

- langchain-experimental: Package holds experimental LangChain code

The packages we will be using are explicitly called out in our requirements.txt

## What Value Does it Provide?

- Provides abstractions for integrating with a large variety of LLM's

- Offers libraries to easily prep documents for Generative AI

- Provides the 'LangChain Expression Language' which enables developers to quickly build LLM workflows

## Import Libraries

In [ ]:
import os
from dotenv import load_dotenv
from typing import Any, Dict, List
load_dotenv()
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.outputs import LLMResult
from langchain_core.messages import BaseMessage

## Define our GPT4o Model 🤖

In [ ]:
model = AzureChatOpenAI(
    azure_deployment="gpt4o",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01"
)

## Create our Callback Handler Class 📞

In [ ]:
class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        print(f"Chain {serialized.get('name')} started")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")

## LangChain Expression Language (chains) 👨‍💻

Below, we will leverage the LangChain Expression Language to do the following:

- Create a [prompt template](https://python.langchain.com/v0.2/docs/how_to/#prompt-templates) which will help format user input into a format that can be passed to a language model

- Leverage the format output from the prompt template to pass along to a [chat model](https://python.langchain.com/v0.2/docs/how_to/#chat-models). In our case, the chat model is GPT4o

- Convert the output into a string leveraging one of LangChains [output parsers](https://python.langchain.com/v0.2/docs/how_to/#output-parsers) called StrOutputParser

- Enable [Callbacks](https://python.langchain.com/v0.2/docs/how_to/callbacks_runtime/) so we can monitor the LLM call

In [ ]:
callbacks = [LoggingHandler()]
prompt = ChatPromptTemplate.from_template("summarize the following Star Wars Movie:{movie}")
chain = prompt | model | StrOutputParser()
chain.invoke({"movie": "A New Hope"}, config={"callbacks": callbacks})

## Making LLM Call's Stateful (ie: chat history) 💬

As mentioned above, one of the nice things about LangChain is the abstractions it provides to solve common technical challenges when building LLM enabled applications, chat history being a common one.

Let's start by defining where we will store out chat history. In our case, we will leverage a local sqlite database.

In [ ]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

Next, let's boiler plate a prompt template that accepts the users message and preferred language as input. We will leverage the sqlite database to save and append the chat history to the prompt

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | model

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

Let's send an initial message introducing ourselves

In [ ]:
runnable_with_history.invoke(
    {"language": "english", "input": "hi im Conner!"},
    config={"configurable": {"session_id": "1"}},
)

Finally, lets follow up and ask what our name is

In [ ]:
runnable_with_history.invoke(
    {"language": "english", "input": "whats my name?"},
    config={"configurable": {"session_id": "1"}},
)